# 整理SLE_GSE255441

[](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE255441)

(未见刊)

肠道微生物群失调在系统性红斑狼疮（SLE）发病机制中的作用仍然难以捉摸。在这里，我们发现从健康小鼠到狼疮小鼠的粪便微生物群移植（FMT）可以改善狼疮样症状。微生物群重建有效地减少了系统类别转换重组，并提高了IGH幼稚同种型。微生物群分析显示，FMT后约氏乳杆菌富集，与嘌呤代谢产物显著相关。重要的是，约翰氏乳杆菌衍生的肌苷是嘌呤代谢的中间代谢产物，通过阻碍B细胞分化和减少肾脏B细胞浸润，有效缓解了狼疮样症状。我们进一步证明肌苷通过ERK-HIF-1α信号通路重新编程B细胞。



kidney samples from pristane-induced C57BL/6j lupus mice treated with saline (n=2) and fecal microbiota transplanation (FMT) from MRL/MpJ mice (n=3).



小鼠 肾脏 SLE小鼠2只 (移植了健康小鼠粪便的)SLE小鼠3只

这数据真好啊，简直完美符合需求

Public on Mar 31, 2024。。。。可恶，注释也没给啊啊啊啊

|||||
|:-|:-|:-|:-|
|GSM8072526|15|replicate 1|scRNA-seq|
|GSM8072527|D2|replicate 2|scRNA-seq|
|GSM8072528|B42|replicate 1|scRNA-seq|
|GSM8072529|B48|replicate 2|scRNA-seq|
|GSM8072530|B50|replicate 3|scRNA-seq|


这么推测，B42,B48,B50是FMT的小鼠

~使用标签转移~别偷懒，改成手动注释，后面可能还要注释亚群

In [ ]:
import sys
from pathlib import Path
p_root = Path("~licanchengup/link/res_publish").expanduser()
None if str(p_root) in sys.path else sys.path.append(str(p_root))

In [ ]:
from  func import *

In [ ]:
df_path = pd.DataFrame({
        'path':Path('GSE255441_RAW').iterdir()
})
df_path = df_path[df_path['path'].apply(lambda x:x.match('*gz'))]
df_path['name'] = df_path['path'].apply(lambda x:x.name)
df_path = df_path.join(
    df_path['name'].str.extract(
        "^(?P<gsm_id>[^_]+)_(?P<sample>[^_]+)_(?P<type>[^_]+)\.\w{3}\.gz")
)
df_path = df_path.pivot(index = 'sample',columns='type',values='path').reset_index()
# assert df_path.index.is_unique,'[Error] not uniuqe'
df_path['treatment'] = 'SLE'
df_path['treatment'] = df_path['treatment'].mask(
    df_path['sample'].str.startswith('B'),
    'SLE_FMT')
df_path

In [ ]:
adata = {}

for i,row in df_path.iterrows():
    print('\r[{}/{}] {}'.format(i+1,df_path.shape[0],row['sample']).ljust(75,'-'))
    _adata = None
    _adata = sc.read_mtx(row['matrix']).T
    
    _df_obs = pd.read_csv(row['barcodes'],header=None,index_col=0,)
    _df_obs.index.name = ''
    _df_var= pd.read_csv(row['features'],header=None,index_col=1,sep='\t')
    _df_var.index.name = ''
    _df_var.columns = 'gene_id,gene_type'.split(',')
    _adata.obs = _df_obs
    _adata.var = _df_var
    # display(_adata.obs.index.is_unique,_adata.var.index.is_unique)
    _adata.obs_names_make_unique()
    _adata.var_names_make_unique()
    
    _adata.obs['sample'] = row['sample']
    _adata.obs['treatment'] = row['treatment']
    # display(_adata,
    #     _adata.obs.head(2),_adata.obs.shape,
    #     _adata.var.head(2),_adata.var.shape
    # )
    
    adata[row['sample']] = _adata
    del _adata,_df_obs,_df_var
    
print("\n[finish]\n".center(100,"-"))

In [ ]:
adata = sc.concat(adata,index_unique=';')
adata.obs['_batch'] = df_apply_merge_field(adata.obs,'{treatment}_{sample}')
adata.obs = adata.obs.loc[:,'_batch,treatment,sample'.split(',')]
display(adata,
    adata.obs.head(2),adata.obs.shape,
    adata.var.head(2),adata.var.shape,
    adata.obs['_batch'].value_counts()
)

In [ ]:
_temp = adata.X
display(_temp[_temp > 0][:10])
del _temp

In [ ]:
display(adata,
    adata.obs.head(2),adata.obs.shape,
    adata.var.head(2),adata.var.shape,
    adata.obs['_batch'].value_counts()
)
h5ad_to_mtx(adata,p_cache.joinpath('disease','SLE_mouse_LN_kidney'))

## `SLE_GSE255441_1_cluster`手动注释

~标签转移~


In [ ]:
df_obs = pd.read_csv('obs_Seurat.csv',index_col=0).rename(
    columns={'X_batch':'_batch'})
display(df_obs.head(2))
adata.obs = adata.obs.loc[:,'_batch,treatment,sample'.split(',')].join(df_obs)
display(adata,
    adata.obs.head(2),adata.obs.shape,
    adata.var.head(2),adata.var.shape,
    adata.obs['_batch'].value_counts()
)
display(adata.obs.shape,df_obs.shape)
adata = adata[adata.obs.index.isin(df_obs.index)]
display(adata.obs.shape,df_obs.shape)

In [ ]:
h5ad_to_mtx(adata,p_cache.joinpath('disease','SLE_mouse_LN_kidney'))

# 与健康小鼠肾合并

In [ ]:
adata_healthy = load_adata(p_cache.joinpath('healthy/m_kidney'))
adata_healthy
adata_merge = sc.concat(
    {'healthy':adata_healthy,'SLE':adata},
    index_unique=';')
adata_merge.obs['seurat_clusters'] = adata_merge.obs.apply(lambda row:
    '{}_{}'.format(row.name.split(';')[-1],int(row['seurat_clusters']))
                                         ,axis=1)

In [ ]:
# obs_seurat_merge.csv 是 merge_cluster.ipynb 的输出
_df_obs = pd.read_csv('obs_seurat_merge.csv',index_col=0)
_df_obs['seurat_clusters'] = _df_obs['seurat_clusters'].astype(str)
adata_merge = adata_merge[_df_obs.index,:]
adata_merge.obs = adata_merge.obs.loc[:,[]].join(_df_obs)
adata_merge.obsm['X_umap'] = adata_merge.obs.loc[:,'UMAP1,UMAP2'.split(',')].to_numpy()
sc.pl.umap(adata_merge,color='seurat_clusters,cell_type'.split(','))

# 走个流程看看

In [ ]:
adata_merge
# Saving count data
adata_merge.layers["counts"] = adata_merge.X.copy()
sc.pp.normalize_total(adata_merge)
sc.pp.log1p(adata_merge)
sc.pp.highly_variable_genes(adata_merge, n_top_genes=2000, batch_key='_batch')
sc.tl.pca(adata_merge)
sc.pl.pca_variance_ratio(adata_merge, n_pcs=50, log=True)

In [ ]:
sc.pp.neighbors(adata_merge)
sc.tl.umap(adata_merge)
# Using the igraph implementation and a fixed number of iterations can be
# significantly faster, especially for larger datasets
sc.tl.leiden(adata_merge)
sc.pl.umap(adata_merge, color=["leiden",'cell_type'])

In [ ]:
display(adata_merge.obs.head(2))
sc.pl.umap(adata_merge,color=['cell_type','status'],legend_loc='on data')